##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 유니코드 문자열

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/beta/tutorials/text/unicode"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" /> TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/r2/tutorials/text/unicode.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/r2/tutorials/text/unicode.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
</table>



Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도 불구하고 공식 영문 문서의 내용과 일치하지 않을 수 있습니다. 이 번역에 개선할 부분이 있다면 tensorflow/docs 깃헙 저장소로 풀 리퀘스트를 보내주시기 바랍니다. 문서 번역이나 리뷰에 참여하려면 docs-ko@tensorflow.org로 메일을 보내주시기 바랍니다

## 소개

자연어 처리 시에 종종 다른 문자 집합들을 갖는 다양한 언어들을 다루게 됩니다. *유니코드*는 대부분 모든 언어들의 문자를 표현할 수 있는 기본적인 인코딩 시스템입니다. 각각의 문자들은 `0` 부터 `0x10FFFF` 까지 중에서 고유한 정수 [코드 포인트](https://en.wikipedia.org/wiki/Code_point)를 사용해서 인코딩 됩니다. *유니코드 문자열*은 0 혹은 그 이상의 코드 포인트들의 시퀀스(sequence)입니다.

이 튜토리얼에서는 텐서플로(Tensorflow)에서 유니코드 문자열들을 표현하는 방법과 표준 문자열의 유니코드류를 사용해서 유니코드 문자를 다루는 방법에 대해서 소개합니다. 스크립트에 기반한 유니코드 문자열을 토큰으로 분리하는 법을 배웁니다. 

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow==2.0.0-alpha0
import tensorflow as tf

## `tf.string` 데이터 타입


  텐서플로(TensorFlow)의 기본 `tf.string` `dtype`은 바이트 문자열들의 텐서를 빌드해 줍니다. 유니코드 문자열들은 기본적으로 utf-8로 인코딩 됩니다.

In [0]:
tf.constant(u"Thanks 😊")

`tf.string` 텐서는 바이트 문자열들을 최소 단위로 다루기 때문에 다양한 길이의 바이트 문자열을 다룰 수 있습니다. 문자열 길이는 텐서 차원(dimensions)에 포함되지 않습니다.

In [0]:
tf.constant([u"You're", u"welcome!"]).shape

## 유니코드 표현

텐서플로(TensorFlow)에서 유니코드 문자열을 표현하기 위한 두 가지 표준 방법들이 있습니다:

* `string` 스칼라 —  코드 포인트들의 시퀀들이  [문자 인코딩](https://en.wikipedia.org/wiki/Character_encoding)을 사용해 인코딩 되는 곳입니다.
* `int32` 벡터 — 각각의 위치들이 개별 코드 포인트를 포함하는 곳입니다.

예를 들어,  아래의 세 가지 방법들이 모두 유니코드 문자열 `"语言处理"`(중국어로 "자연어 처리"를 의미함)을 표현하는 방식입니다.

In [0]:
# UTF-8 로 인코드된 문자열 스칼라로 표현한, 유니코드 문자열입니다.
text_utf8 = tf.constant(u"语言处理")
text_utf8

In [0]:
# UTF-16-BE 로 인코드된 문자열 스칼라로 표현한, 유니코드 문자열입니다.
text_utf16be = tf.constant(u"语言处理".encode("UTF-16-BE"))
text_utf16be

In [0]:
# 유니코드 코드 포인트들의 벡터로 표현한, 유니코드 문자열입니다.
text_chars = tf.constant([ord(char) for char in u"语言处理"])
text_chars

### 표현식 사이의 변환

텐서플로(TensorFlow)는 다양한 표현식들 사이에서 변환할 수 있는 연산들(operations)을 제공합니다.

* `tf.strings.unicode_decode`: 인코드된 문자열 스칼라를 코드 포인트들의 벡터로 변환합니다.
* `tf.strings.unicode_encode`: 코드 포인트들의 벡터를 인코드된 문자열 스칼라로 변환합니다.
* `tf.strings.unicode_transcode`: 인코드된 문자열 스칼라를 다른 인코딩으로 변환합니다.

In [0]:
tf.strings.unicode_decode(text_utf8,
                          input_encoding='UTF-8')

In [0]:
tf.strings.unicode_encode(text_chars,
                          output_encoding='UTF-8')

In [0]:
tf.strings.unicode_transcode(text_utf8,
                             input_encoding='UTF8',
                             output_encoding='UTF-16-BE')

### 배치 차원

다양한 문자열들을 디코딩 할 때,  문자열 속 문자들의 수는 동일하지 않습니다. 반환 값은 가장 안 쪽 차원의 길이가 문자열 속 문자들의 수에 따라  다양한, [`tf.RaggedTensor`](../../guide/ragged_tensors.ipynb) 입니다.

In [0]:
# UTF-8로 인코드된 문자열로 표현된, 유니코드 문자열 배치입니다. 
batch_utf8 = [s.encode('UTF-8') for s in
              [u'hÃllo',  u'What is the weather tomorrow',  u'Göödnight', u'😊']]
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8,
                                               input_encoding='UTF-8')
for sentence_chars in batch_chars_ragged.to_list():
  print(sentence_chars)

`tf.RaggedTensor`를 바로 사용하거나, 패딩(padding)을 사용해 `tf.Tensor`로 변환하거나,  `tf.RaggedTensor.to_tensor` 와 `tf.RaggedTensor.to_sparse` 방법을 사용해 `tf.SparseTensor`로 변환할 수 있습니다.

In [0]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

In [0]:
batch_chars_sparse = batch_chars_ragged.to_sparse()

같은 길이의 문자열 인코딩 시, `tf.Tensor` 가 입력값으로 사용됩니다.

In [0]:
tf.strings.unicode_encode([[99, 97, 116], [100, 111, 103], [ 99, 111, 119]],
                          output_encoding='UTF-8')

다양한 길이의 문자열 인코딩 시, `tf.RaggedTensor` 가 입력값으로 쓰입니다.

In [0]:
tf.strings.unicode_encode(batch_chars_ragged, output_encoding='UTF-8')

패딩되거나 희소한 포맷 상의 다양한 문자열을 갖는 텐서가 있다면, `unicode_encode` 호출 전에 `tf.RaggedTensor`로 바꿉니다.

In [0]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8')

In [0]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1),
    output_encoding='UTF-8')

## 유니코드 연산

### 문자 길이

`tf.strings.length` 연산은 어느 정도의 길이가 연산 되어야 하는지 표시하는 `unit` 인자를 가집니다.  `unit`의 기본 단위는 `"바이트(BYTE)"`이지만, 인코드된 `문자열` 속 유니코드 코드 포인트들의 수를 결정하기 위한 `"UTF8_CHAR"`나 `"UTF16_CHAR"`같은 다양한 값들을 설정할 수 있습니다.

In [0]:
# 마지막 문자는 UTF8에서 4바이트를 차지합니다.
thanks = u'Thanks 😊'.encode('UTF-8')
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} bytes; {} UTF-8 characters'.format(num_bytes, num_chars))

### 문자 서브문자열

이와 유사하게, `tf.strings.substr` 연산은  "`unit`" 인자를 받고, "`unit`"를 사용해서  "`pos`" 와  "`len`" 인자를 포함한 오프셋(offset)의 종류를 결정합니다.

In [0]:
# 기본: unit='BYTE'. len=1과 함께, 싱글 바이트를 반환합니다.
tf.strings.substr(thanks, pos=7, len=1).numpy()

In [0]:
# unit='UTF8_CHAR'인 상황에서, 4바이트인 싱글 문자를 반환합니다.
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

### 유니코드 문자열 분리

`tf.strings.unicode_split` 연산은 유니코드 문자열을 개별 문자들의 서브 문자열로 분리합니다.

In [0]:
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

### 문자를 위한 바이트 오프셋(offset)

`tf.strings.unicode_decode`를 통해 생성된 문자 텐서들을 정렬하기 위해, 각 문자들이 어디서 시작하는지 알기 위한 오프셋(offset)을 아는 것이 유용합니다. `tf.strings.unicode_decode_with_offsets`은 각각 문자의 시작 오프셋을 포함한 두 번째 텐서를 반환함을 제외하면, `unicode_decode`와 비슷합니다.

In [0]:
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u"🎈🎉🎊", 'UTF-8')

for (codepoint, offset) in zip(codepoints.numpy(), offsets.numpy()):
  print("At byte offset {}: codepoint {}".format(offset, codepoint))

## 유니코드 스크립트(Script)

유니코드 코드 포인트는 코드 포인트들의 [스크립트](https://en.wikipedia.org/wiki/Script_%28Unicode%29)라 알려져 있는 싱글 컬렉션(Collection)에 속합니다. 문자의 스크립트는 문자가 어떤 언어일지 결정하는 것을 도와줍니다. 예를 들어, 키릴 스크립트 상의 'Б' 은  (러시아어나 우크라이나어 같은) 슬라브 언어와 비슷한 문자를 포함하는 현대 텍스트를 가리키는 것으로 알려져 있습니다.

텐서플로(Tensorflow)는 어떤 스크립트가 주어진 코드 포인트를 사용하는지 판별하기 위한 `tf.strings.unicode_script` 연산을 제공합니다. 스크립트 코드들은 [International Components for
Unicode](http://site.icu-project.org/home) (ICU) [`UScriptCode`](http://icu-project.org/apiref/icu4c/uscript_8h.html) 의 값과 일치하는 `int32` 값입니다.

In [0]:
uscript = tf.strings.unicode_script([33464, 1041])  # ['芸', 'Б']

print(uscript.numpy())  # [17, 8] == [USCRIPT_HAN, USCRIPT_CYRILLIC]

`tf.strings.unicode_script` 연산은 다차원적 `tf.Tensor`들이나  코드 포인트들의 `tf.RaggedTensor`들에 적용될 수 있습니다:

In [0]:
print(tf.strings.unicode_script(batch_chars_ragged))

## 예제: 간단한 분절

분절은 단어-와 같은 단위로 텍스트를 나누는 방식입니다. 띄어쓰기가 단어를 나누는 구분자로 사용되는 경우는 쉽게 나눠질 수 있으나, 띄어쓰기를 사용하지 않는 (중국어나 일본어 같은) 언어들이나 긴 합성어를 사용하는 (독일어 같은) 언어들은 의미를 분석하기 위한 분리 과정이 꼭 필요합니다. 웹 텍스트 상에서, 다른 언어들과 스크립트들이 종종 혼재하기도 합니다. 예를 들어,  (New York Stock Exchange)를 의미하는 "NY株価" 에서 영어와 중국어가 섞여 있습니다.

대강의 단어 경계선을 통해서 (ML 모델 사용 없이)  스크립트를 대략적으로 분절할 수 있습니다. 위에서 언급된 "NY株価" 예시에 쓰일 수 있고,
다양한 스크립트들의 띄어쓰기 문자들이 USCRIPT_COMMON(실제 텍스트상의 스크립트 코드와 다른 특별한 스크립트 코드를 의미함)로 모두 분류되면서 띄어쓰기를 사용하는 대부분의 언어들에서도 역시 적용됩니다. 


In [0]:
# dtype: string; shape: [num_sentences]
#
# 진행할 문장들 입니다. 이 라인을 수정해서 다른 입력값으로 시도해 보세요!
sentence_texts = [u'Hello, world.', u'世界こんにちは']

우선, 문장들을 문자 코드 포인트로 디코드하고 각각의 문자들을 위한 스크립트  식별자를 찾습니다.

In [0]:
# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_codepoint[i, j]는
# i번째 문장 안의 j번째 문자을 위한 코드 포인트 입니다.
sentence_char_codepoint = tf.strings.unicode_decode(sentence_texts, 'UTF-8')
print(sentence_char_codepoint)

# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_codepoint[i, j]는 
# i번째 문장 속 j번째 문자의 유니코드 스크립트 입니다.
sentence_char_script = tf.strings.unicode_script(sentence_char_codepoint)
print(sentence_char_script)

다음은, 단어 경계를 표시하기 위한 스크립트 식별자를 사용합니다. 문장 시작 부분과, 이전 문자와 스크립트가 다른 문자에 단어 분리선을 추가합니다.

In [0]:
# dtype: bool; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_starts_word[i, j] 는 
# i번째 문장 속 j번째 문자가 단어의 시작이면 True 입니다.
sentence_char_starts_word = tf.concat(
    [tf.fill([sentence_char_script.nrows(), 1], True),
     tf.not_equal(sentence_char_script[:, 1:], sentence_char_script[:, :-1])],
    axis=1)

# dtype: int64; shape: [num_words]
#
# word_starts[i]은 (모든 문장들 속 문자들의 평탄화된(flattened) 리스트에서)
# i번째 단어의 시작인 문자 인덱스 입니다.
word_starts = tf.squeeze(tf.where(sentence_char_starts_word.values), axis=1)
print(word_starts)

모든 배치들로부터의 단어 리스트를 갖는 `RaggedTensor` 를 빌드하기 위해 시작 오프셋(offsets)을 사용합니다.

In [0]:
# dtype: int32; shape: [num_words, (num_chars_per_word)]
#
# word_char_codepoint[i, j]은 
# i번째 단어 안의 j번째 문자를 위한 코드 포인트 입니다.
word_char_codepoint = tf.RaggedTensor.from_row_starts(
    values=sentence_char_codepoint.values,
    row_starts=word_starts)
print(word_char_codepoint)

마지막으로, 단어 코드 포인트 `RaggedTensor`를 분절한 후 문장 속으로 집어 넣습니다.

In [0]:
# dtype: int64; shape: [num_sentences]
#
# sentence_num_words[i] 는 i번째 문장 속 단어들의 수 입니다.
sentence_num_words = tf.reduce_sum(
    tf.cast(sentence_char_starts_word, tf.int64),
    axis=1)

# dtype: int32; shape: [num_sentences, (num_words_per_sentence), (num_chars_per_word)]
#
# sentence_word_char_codepoint[i, j, k]는 i번째 문장 안의 j번째 단어 안의 
# k번째 문자를 위한 코드 포인트 입니다.
sentence_word_char_codepoint = tf.RaggedTensor.from_row_lengths(
    values=word_char_codepoint,
    row_lengths=sentence_num_words)
print(sentence_word_char_codepoint)

최종 결과를 읽기 쉽게 utf-8 문자열로 다시 인코드 합니다.

In [0]:
tf.strings.unicode_encode(sentence_word_char_codepoint, 'UTF-8').to_list()